# Fine-tuning Sandbox

Ce notebook contient des exemples de code pour le fine-tuning de grands modèles de langage (Large Language Models, LLMs).

In [1]:
# Installation des bibliothèques nécessaires
!pip install peft
!pip install evaluate


## Importation des Bibliothèques

Importation des bibliothèques nécessaires pour le traitement des données et l'entraînement du modèle.


In [2]:
# Importation des bibliothèques essentielles pour le traitement des données et le machine learning
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split



# Bibliothèque pour l'évaluation des modèles
import evaluate


# Chargement des données

In [3]:
# Chemin le fichier
file_path = 'IMDB_Dataset.csv'

# Charger les données depuis le fichier CSV
df = pd.read_csv(file_path)
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
# Conversion des étiquettes en format numérique
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df = df.drop(['sentiment'], axis=1) 
df.head()


,review,label
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:

# Séparer les données en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [6]:

# Sélectionner un sous-ensemble aléatoire de N échantillons pour l'entraînement et le test
N = 1000
train_subset = train_df.sample(n=N, random_state=42)
test_subset = test_df.sample(n=N, random_state=42)


In [7]:

# Créer les datasets Hugging Face
train_dataset = Dataset.from_pandas(train_subset)
test_dataset = Dataset.from_pandas(test_subset)



In [8]:
train_dataset

Dataset({
    features: ['review', 'label', '__index_level_0__'],
    num_rows: 1000
})

In [9]:
test_dataset

Dataset({
    features: ['review', 'label', '__index_level_0__'],
    num_rows: 1000
})

# Finetuning complet

DistilBERT (de "Distilled BERT") est une version plus petite et plus rapide du modèle BERT (Bidirectional Encoder Representations from Transformers) développé par Hugging Face. Il offre des performances proches de BERT avec une taille réduite et une vitesse accrue.


In [11]:
import torch
# Importe PyTorch, une bibliothèque pour les réseaux de neurones profonds et l'apprentissage automatique.

import pandas as pd
# Importe pandas, une bibliothèque pour la manipulation et l'analyse de données, souvent utilisée pour manipuler des tableaux de données.

from datasets import Dataset, DatasetDict
# Importe 'Dataset' et 'DatasetDict' de la bibliothèque 'datasets' de Hugging Face, utilisés pour gérer et préparer des ensembles de données pour l'entraînement de modèles.

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
# Importe plusieurs classes de la bibliothèque 'transformers' de Hugging Face:
# - DistilBertTokenizer : pour convertir du texte en tokens compréhensibles par DistilBERT.
# - DistilBertForSequenceClassification : une version de DistilBERT préparée pour la classification de séquences (par exemple, classification de texte).
# - TrainingArguments : pour configurer les paramètres d'entraînement.
# - Trainer : pour orchestrer le processus d'entraînement du modèle.

import torch.optim as optim
# Importe le sous-module 'optim' de PyTorch, qui contient divers algorithmes d'optimisation utilisés pour mettre à jour les poids du réseau pendant l'entraînement, tels que SGD, Adam, etc.



In [12]:
# Paramètres d'entraînement
model_checkpoint = "distilbert-base-uncased"  # Modèle distilbert pré-entraîné
lr = 2e-5  # Taux d'apprentissage
batch_size = 8  # Taille du lot
num_epochs = 3  # Nombre d'époques


Dans le domaine du traitement du langage naturel (NLP), le "padding" joue un rôle crucial dans la préparation des données pour l'entraînement de modèles. Voici quelques détails clés :

Concept de Padding en NLP
Uniformité des Longueurs de Séquences : Les modèles de NLP, en particulier ceux basés sur des réseaux de neurones, nécessitent que toutes les entrées (phrases, paragraphes, etc.) dans un lot de données (batch) aient la même longueur pour un traitement efficace. Cependant, dans la réalité, les séquences de mots varient en longueur.
Ajout de Valeurs de Remplissage : Pour résoudre ce problème, le padding est utilisé pour "remplir" les séquences plus courtes avec des valeurs neutres (souvent des zéros ou un token spécifique) pour qu'elles correspondent à la longueur de la séquence la plus longue dans le lot.

In [14]:
# Charger le tokenizer et le modèle pré-entraîné
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_dataset['review'], truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dataset['review'], truncation=True, padding=True, return_tensors='pt')


In [15]:
# Étape 2 : Création des datasets PyTorch
# Étape 2 : Création des datasets PyTorch
train_datasetb = Dataset.from_dict({
    # 'input_ids' : Contient les identifiants de tokens obtenus après le processus de tokenisation du texte. 
    # Ces identifiants sont nécessaires pour que le modèle comprenne et traite le texte.
    'input_ids': train_encodings['input_ids'],

    # 'attention_mask' : Un masque d'attention qui indique au modèle quels tokens dans 'input_ids' sont significatifs 
    # (c'est-à-dire non-padding) et doivent être pris en compte lors du traitement.
    'attention_mask': train_encodings['attention_mask'],

    # 'labels' : Les étiquettes associées à chaque exemple d'entraînement. Ces étiquettes sont utilisées pour l'apprentissage supervisé,
    # où le modèle apprend à associer les entrées à ces étiquettes correctes.
    # 'torch.tensor()' est utilisé pour convertir la liste des étiquettes en tensor PyTorch, 
    # ce qui est nécessaire pour le traitement avec PyTorch.
    'labels': torch.tensor(train_dataset['label'])
})

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_dataset['label'])
})

# Création d'un DatasetDict
datasetb = DatasetDict({
    'train': train_datasetb,
    'test': test_datasetb
})

In [16]:
# Création d'un modèle de classification de séquences basé sur DistilBERT
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Affichage du nombre de paramètres du modèle
num_params = sum(p.numel() for p in model.parameters())
print(f"Nombre de paramètres du modèle : {num_params}")

# Afficher l'architecture du modèle BERT
print(model)


Nombre de paramètres du modèle : 66955010
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            

In [18]:
# Étape 4 : Définition des paramètres d'entraînement
# Configuration de l'optimiseur pour l'entraînement du modèle
optimizer = optim.AdamW(model.parameters(), lr=lr)
# Définition de la fonction de perte pour l'entraînement du modèle
loss_fn = torch.nn.CrossEntropyLoss()

In [19]:
# Importer la métrique pour l'évaluation
accuracy = evaluate.load("accuracy")

In [20]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy_value}


In [21]:

# Définir la correspondance entre les identifiants de classe et les étiquettes
id2label = {0: "Négatif", 1: "Positif"}
label2id = {"Négatif": 0, "Positif": 1}


In [22]:
liste_texte = [
    "This movie was a masterpiece.",
    "I was blown away by the acting.",
    "It's a classic that everyone should watch.",
    "The plot was confusing and hard to follow.",
    "The special effects were top-notch.",
    "I couldn't stop laughing throughout the movie.",
    "The soundtrack was incredible.",
    "It's a total waste of time.",
    "I'm still thinking about that ending.",
    "I wouldn't recommend it to anyone."
]


print("Prédictions du modèle non entraîné:")
print("-------------------------------------")
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt")
    # Calculer les logits
    logits = model(inputs).logits
    # Convertir les logits en étiquette
    predictions = torch.argmax(logits)

    print(texte + " - " + id2label[predictions.tolist()])

Prédictions du modèle non entraîné:
-------------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Positif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Négatif
I couldn't stop laughing throughout the movie. - Positif
The soundtrack was incredible. - Négatif
It's a total waste of time. - Positif
I'm still thinking about that ending. - Positif
I wouldn't recommend it to anyone. - Positif


In [24]:
# Étape 5 : Définition des arguments d'entraînement avec Transformers

training_args = TrainingArguments(
    output_dir=model_checkpoint + "-fullfinetuned-text-classification",  # Le répertoire où les résultats de l'entraînement seront sauvegardés
    learning_rate=lr,  # Taux d'apprentissage pour l'optimisation du modèle
    per_device_train_batch_size=batch_size,  # Taille du lot d'entraînement par périphérique (GPU ou CPU)
    per_device_eval_batch_size=batch_size,  # Taille du lot d'évaluation par périphérique (GPU ou CPU)
    num_train_epochs=num_epochs,  # Nombre d'époques d'entraînement (combien de fois le modèle parcourt l'ensemble de données)
    weight_decay=0.01,  # Terme de régularisation pour contrôler le poids de la pénalisation dans la fonction de perte
    evaluation_strategy="epoch",  # Stratégie d'évaluation, ici "epoch" signifie évaluer à la fin de chaque époque
    save_strategy="epoch",  # Stratégie de sauvegarde du modèle, ici "epoch" signifie sauvegarder le modèle à la fin de chaque époque
    load_best_model_at_end=True  # Charger le meilleur modèle à la fin de l'entraînement
)


In [25]:
# Étape 6 : Entraînement du modèle avec Trainer de Transformers
trainerfull = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasetb['train'],
    eval_dataset=datasetb['test'],
    compute_metrics=compute_metrics  # Utiliser la fonction de calcul des métriques

)


In [26]:
# Entraînement du modèle
trainerfull.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.309968,0.891000
2,No log,0.365536,0.898000
3,No log,0.402130,0.901000


TrainOutput(global_step=375, training_loss=0.22651810709635417, metrics={'train_runtime': 275.285, 'train_samples_per_second': 10.898, 'train_steps_per_second': 1.362, 'total_flos': 397402195968000.0, 'train_loss': 0.22651810709635417, 'epoch': 3.0})

In [27]:

# Sauvegarde du modèle
trainerfull.save_model("trainer_full")

In [28]:
# Passer le modèle en mode 'mps' (pour Mac, vous pouvez également utiliser 'cpu')
model.to('mps')

# Afficher les prédictions du modèle entraîné
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt").to("mps") # Passage en mode 'mps' (ou 'cpu' en alternative pour Mac)

    # Obtenir les logits du modèle pour le texte
    logits = model(inputs).logits

    # Prédire la classe en choisissant l'indice du logit le plus élevé
    predictions = torch.max(logits, 1).indices

    # Afficher le texte et l'étiquette prédite
    print(texte + " - " + id2label[predictions.tolist()[0]])


Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif


# Layer freezing finetuning

In [29]:
# Charger le tokenizer et le modèle pré-entraîné
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_dataset['review'], truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dataset['review'], truncation=True, padding=True, return_tensors='pt')


In [30]:
train_datasetb = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(train_dataset['label'])
})

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_dataset['label'])
})


In [31]:
# Modèle DistilBERT pour la classification de séquences
modelf = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Geler certaines couches du modèle
for param in modelf.base_model.parameters():
    param.requires_grad = False

In [32]:
# Définir les arguments d'entraînement pour le Trainer de Hugging Face
training_args = TrainingArguments(
    # Répertoire où les résultats d'entraînement (modèles, configurations, etc.) seront sauvegardés.
    output_dir="./freezdistilbert_finetuned",

    # Nombre d'exemples par lot (batch) pour l'entraînement, défini pour chaque périphérique (GPU/CPU).
    per_device_train_batch_size=8,

    # Nombre d'exemples par lot pour l'évaluation, défini pour chaque périphérique.
    per_device_eval_batch_size=8,

    # Stratégie d'évaluation à utiliser. Ici, "steps" signifie que l'évaluation aura lieu à intervalles réguliers de pas d'entraînement.
    evaluation_strategy="steps",

    # Nombre de pas d'entraînement à effectuer avant de réaliser une évaluation.
    eval_steps=100,

    # Nombre de pas d'entraînement après lesquels le modèle sera sauvegardé.
    save_steps=100,

    # Nombre total d'époques d'entraînement à réaliser.
    num_train_epochs=3,

    # Taux d'apprentissage initial à utiliser pour l'optimiseur.
    learning_rate=2e-5,

    # Limite du nombre total de sauvegardes de checkpoints à conserver. 
    # Ici, seul le checkpoint le plus récent sera conservé.
    save_total_limit=1
)


In [33]:
# Importer la métrique d'exactitude pour l'évaluation

accuracy = evaluate.load("accuracy")

In [35]:
# Fonction pour calculer l'exactitude
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    # Calcul de l'exactitude
    accuracy = np.mean(predictions == labels)

    return {"accuracy": accuracy}

In [37]:
from transformers import DataCollatorWithPadding  # Assurez-vous d'avoir cette ligne d'importation


# Créer un Trainer
trainerfreezing = Trainer(
    model=modelf,  # Utilisez le modèle DistilBERT que vous avez défini précédemment
    args=training_args,  # Les paramètres d'entraînement définis précédemment
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # Gère le padding des données lors de l'entraînement
    compute_metrics=compute_metrics,  # Fonction pour calculer les métriques d'évaluation
    train_dataset=train_datasetb,  # L'ensemble de données d'entraînement
    eval_dataset=test_datasetb  # L'ensemble de données d'évaluation
)

In [38]:
# Entraîner le modèle
trainerfreezing.train()

Step,Training Loss,Validation Loss,Accuracy
100,No log,0.322988,0.878000
200,No log,0.358135,0.887000
300,No log,0.389590,0.898000


TrainOutput(global_step=375, training_loss=0.24811503092447917, metrics={'train_runtime': 274.1337, 'train_samples_per_second': 10.944, 'train_steps_per_second': 1.368, 'total_flos': 397402195968000.0, 'train_loss': 0.24811503092447917, 'epoch': 3.0})

In [39]:
# Passer le modèle en mode 'mps' (pour Mac, vous pouvez également utiliser 'cpu')
modelf.to('mps')

# Afficher les prédictions du modèle entraîné
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt").to("mps") # Passage en mode 'mps' (ou 'cpu' en alternative pour Mac)

    # Obtenir les logits du modèle pour le texte
    logits = modelf(inputs).logits

    # Prédire la classe en choisissant l'indice du logit le plus élevé
    predictions = torch.max(logits, 1).indices

    # Afficher le texte et l'étiquette prédite
    
    # Afficher le texte et l'étiquette prédite
    print(texte + " - " + id2label[predictions.tolist()[0]])

Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif


# Finetuning LoRA

In [41]:
# Importation des outils spécifiques aux modèles transformers
from transformers import (
    AutoTokenizer,  # Pour le tokenizing des textes
    AutoModelForSequenceClassification,  # Pour charger un modèle pré-entraîné pour la classification de séquences
    TrainingArguments,  # Pour définir les paramètres d'entraînement
    Trainer,  # Pour entraîner le modèle
    DataCollatorWithPadding  # Pour gérer le padding des séquences lors de l'entraînement
)
# Importation des bibliothèques pour le fine-tuning avec PEFT (Parameter-Efficient Fine-tuning)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [42]:

# Définir le modèle que vous souhaitez utiliser pour la classification de séquences. Vous pouvez choisir un modèle pré-entraîné, comme 'distilbert-base-uncased' ou 'roberta-base'.

model_checkpoint = 'distilbert-base-uncased'

# Générer un modèle de classification à partir du modèle pré-entraîné
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Preprocessing

In [44]:
# Créer un tokenizer pour le modèle
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
tokenizer.pad_token
# Ajouter un jeton de padding s'il n'existe pas
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))



In [45]:
# Créer une fonction pour tokenizer les données
def tokenize_function(examples):
    text = examples["review"]
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        padding=True,  # Ajout du padding directement ici
        truncation=True,
        max_length=512
    )
    return tokenized_inputs

In [46]:
# Tokenizer les ensembles d'entraînement et de test
# Créer un DatasetDict
# Tokenizer les ensembles d'entraînement et de test et créer un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['review', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [47]:
# Créer un DataCollator pour gérer le padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluation du modèle

In [48]:
# Configuration du modèle LoRA (Low-Rank Adaptation) pour le fine-tuning

# Type de tâche, ici "SEQ_CLS" signifie classification de séquences
peft_config = LoraConfig(
    task_type="SEQ_CLS",  # Type de tâche du modèle

    # Facteur de régularisation "r" pour LoRA
    r=4,

    # Paramètre d'alpha pour LoRA, contrôlant l'importance de la régularisation LoRA
    lora_alpha=32,

    # Taux de dropout pour LoRA, qui détermine la probabilité de désactivation aléatoire des connexions
    lora_dropout=0.01,

    # Modules cibles pour l'application de la régularisation LoRA, ici uniquement le module 'q_lin' est ciblé
    target_modules=['q_lin']
)


In [49]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [50]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [51]:
# Hyperparamètres

# Taux d'apprentissage (learning rate) - C'est la vitesse à laquelle le modèle apprend
lr = 1e-3

# Taille du lot (batch size) - Le nombre d'exemples de données utilisés pour chaque mise à jour de poids du modèle
batch_size = 4

# Nombre d'époques (epochs) - Le nombre de fois que le modèle parcourt l'ensemble de données complet lors de l'entraînement
num_epochs = 3


In [52]:
# Définition des arguments d'entraînement

# Répertoire de sortie où les résultats de l'entraînement seront sauvegardés
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",  # Chemin du répertoire de sortie

    # Taux d'apprentissage (learning rate) pour l'optimisation du modèle
    learning_rate=lr,

    # Taille du lot (batch size) d'entraînement par périphérique (GPU ou CPU)
    per_device_train_batch_size=batch_size,

    # Taille du lot (batch size) d'évaluation par périphérique (GPU ou CPU)
    per_device_eval_batch_size=batch_size,

    # Nombre d'époques d'entraînement (combien de fois le modèle parcourt l'ensemble de données)
    num_train_epochs=num_epochs,

    # Terme de régularisation pour contrôler le poids de la pénalisation dans la fonction de perte
    weight_decay=0.01,

    # Stratégie d'évaluation, ici "epoch" signifie évaluer à la fin de chaque époque
    evaluation_strategy="epoch",

    # Stratégie de sauvegarde du modèle, ici "epoch" signifie sauvegarder le modèle à la fin de chaque époque
    save_strategy="epoch",

    # Charger le meilleur modèle à la fin de l'entraînement
    load_best_model_at_end=True
)


In [53]:
# Création de l'objet "trainer" pour entraîner le modèle

# Le modèle à entraîner
trainer = Trainer(
    model=model,  # Le modèle que vous avez configuré précédemment

    # Les arguments d'entraînement définis précédemment
    args=training_args,

    # L'ensemble de données d'entraînement tokenisé
    train_dataset=tokenized_dataset["train"],

    # L'ensemble de données de validation tokenisé
    eval_dataset=tokenized_dataset["test"],

    # Le tokenizer utilisé pour le prétraitement
    tokenizer=tokenizer,

    # Le "data_collator" qui gère le padding dynamique des exemples dans chaque lot pour qu'ils aient la même longueur
    data_collator=data_collator,

    # La fonction de calcul des métriques pour évaluer le modèle
    compute_metrics=compute_metrics
)




In [54]:
# Entraînement du modèle
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.429260,0.873000
2,0.339000,0.585174,0.877000
3,0.339000,0.702090,0.875000


TrainOutput(global_step=750, training_loss=0.2660241495768229, metrics={'train_runtime': 230.0579, 'train_samples_per_second': 13.04, 'train_steps_per_second': 3.26, 'total_flos': 403199004672000.0, 'train_loss': 0.2660241495768229, 'epoch': 3.0})

### Generate prediction

In [55]:
# Passer le modèle en mode 'mps' (pour Mac, vous pouvez également utiliser 'cpu')
model.to('mps')

# Afficher les prédictions du modèle entraîné
print("Prédictions du modèle entraîné :")
print("---------------------------------")

# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt").to("mps") # Passage en mode 'mps' (ou 'cpu' en alternative pour Mac)

    # Obtenir les logits du modèle pour le texte
    logits = model(inputs).logits

    # Prédire la classe en choisissant l'indice du logit le plus élevé
    predictions = torch.max(logits, 1).indices

    # Afficher le texte et l'étiquette prédite
    print(texte + " - " + id2label[predictions.tolist()[0]])


Prédictions du modèle entraîné :
---------------------------------
This movie was a masterpiece. - Positif
I was blown away by the acting. - Négatif
It's a classic that everyone should watch. - Positif
The plot was confusing and hard to follow. - Négatif
The special effects were top-notch. - Positif
I couldn't stop laughing throughout the movie. - Négatif
The soundtrack was incredible. - Positif
It's a total waste of time. - Négatif
I'm still thinking about that ending. - Négatif
I wouldn't recommend it to anyone. - Négatif
